<a href="https://colab.research.google.com/github/dmachlanski/iads-summer-school-causality-2023/blob/main/labs/Exercise_JOBS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Causal Inference - Exercise (JOBS)

This is an optional exercise for anyone going quicker throught the labs than anticipated. We leave here very little instructions to let people approach the problem however they want, though it can be tackled in a similar manner as the IHDP one.

This dataset, proposed by [A. Smith and E. Todd (2005)](https://ideas.repec.org/a/eee/econom/v125y2005i1-2p305-353.html), is a combination of the experiment done by [LaLonde (1986)](https://www.jstor.org/stable/1806062) as part of the National Supported Work Program (NSWP) and observational data from the Panel Study of Income Dynamics (PSID) [(Dehejia and Wahba 2002)](https://ideas.repec.org/a/tpr/restat/v84y2002i1p151-161.html). Overall, the data captures people’s basic characteristics, whether they received a job training from NSWP (the treatment), and finally their employment status (the outcome).

JOBS consists of 17 background features (mixture of continuous and binary), a binary treatment, and a binary outcome. It includes 3,212 samples, of which 297/2,915 are treated/control units.

The dataset can be accessed [here](https://github.com/dmachlanski/iads-summer-school-causality-2023/tree/main/labs/data). URLs to training and testing parts of the data:
- https://github.com/dmachlanski/iads-summer-school-causality-2023/raw/main/labs/data/jobs_train.npz
- https://github.com/dmachlanski/iads-summer-school-causality-2023/raw/main/labs/data/jobs_test.npz

Contrary to previous examples, a different set of evaluation mertics is usually used against JOBS. These are: a) Average Treatment effect on the Treated (ATT), and b) policy risk. They are formally defined as follows.

Given a set of treated subjects $T$ that are part of sample $E$ coming from an experimental study, and a set of control group $C$, define ATT as:

$$ATT = \frac{1}{|T|}\sum \limits_{i \in T}\mathcal{Y}^{(i)} - \frac{1}{|C \cap E|}\sum \limits_{i \in C \cap E}\mathcal{Y}^{(i)}$$

The error on ATT is then defined as the absolute difference between the true and predicted ATT:

$$\epsilon_{ATT} = \left| ATT - \frac{1}{|T|}\sum \limits_{i \in T} (\hat{y}_1^{(i)}-\hat{y}_0^{(i)}) \right|$$

Policy risk can be defined as:

$$\mathcal{R}_{pol} = 1 - (\mathbb{E}\left [ \mathcal{Y}_1|\pi(x)=1 \right ] \mathcal{P}(\pi(x)=1) + \mathbb{E}\left [ \mathcal{Y}_0|\pi(x)=0 \right ] \mathcal{P}(\pi(x)=0))$$

Where $\mathbb{E}[.]$ denotes mathematical expectation and policy $\pi$ becomes $\pi(x)=1$ if $\hat{y}_1 - \hat{y}_0 > 0$; $\pi(x)=0$ otherwise.

The code provided below computes both $\epsilon_{ATT}$ and $\mathcal{R}_{pol}$ given predicted effects. Apart from the background (X), treatment (T) and outcome (Y) covariates, the dataset also contains information whether a unit comes from an experimental study (E), which is required to compute the metrics as well.

In [ ]:
def calc_stats(y, t, e, te_pred):
    att = np.mean(y[t > 0]) - np.mean(y[(1 - t + e) > 1])

    att_pred = np.mean(te_pred[(t + e) > 1])
    bias_att = att_pred - att

    policy_value = policy_val(y, t, e, te_pred[e > 0])

    eps_att = np.abs(bias_att)
    r_pol = 1 - policy_value
    return eps_att, r_pol

def policy_val(y, t, e, te_pred):
    t_e = t[e > 0]
    y_e = y[e > 0]

    if np.any(np.isnan(te_pred)):
        return np.nan, np.nan

    policy = te_pred > 0.0
    treat_overlap = (policy == t_e) * (t_e > 0)
    control_overlap = (policy == t_e) * (t_e < 1)

    if np.sum(treat_overlap) == 0:
        treat_value = 0
    else:
        treat_value = np.mean(y_e[treat_overlap])

    if np.sum(control_overlap) == 0:
        control_value = 0
    else:
        control_value = np.mean(y_e[control_overlap])

    pit = np.mean(policy)
    policy_value = pit * treat_value + (1 - pit) * control_value

    return policy_value